In [1]:
import vtk
from vtkmodules.vtkFiltersGeneral import vtkTransformFilter
from vtkmodules.vtkCommonTransforms import vtkTransform
from vtkmodules.vtkCommonCore import vtkLookupTable


sphere = vtk.vtkSphereSource()
sphere.SetCenter(0.0,0.0,0.0)
sphere.SetRadius(5)
sphere.SetPhiResolution( 50 )
sphere.SetThetaResolution( 50 )

tr = vtkTransform()
tr.Scale(2, 0.5, 0.75)

transFilter = vtkTransformFilter()
transFilter.SetInputConnection(sphere.GetOutputPort())
transFilter.SetTransform(tr)

vplane = vtk.vtkPlane()
vplane.SetOrigin(0,0,0)
vplane.SetNormal(1.2,1.4,2)

clipper = vtk.vtkClipPolyData()
clipper.SetInputConnection( transFilter.GetOutputPort())
clipper.SetClipFunction( vplane )
clipper.SetValue(0)

normals = vtk.vtkPolyDataNormals()
normals.SetInputConnection(clipper.GetOutputPort())
# normals.FlipNormalsOn()
normals.FlipNormalsOff()
# normals.SetFeatureAngle(60.0)

cone = vtk.vtkConeSource()
cone.SetResolution( 3 )

glyph = vtk.vtkGlyph3D()
glyph.SetInputConnection( normals.GetOutputPort())
glyph.SetSourceConnection( cone.GetOutputPort())
glyph.SetVectorModeToUseNormal()
glyph.SetScaleModeToScaleByVector()
glyph.SetScaleFactor(0.4)

glyph_mapper = vtk.vtkPolyDataMapper()
glyph_mapper.SetInputConnection(glyph.GetOutputPort())

glyph_actor = vtk.vtkActor()
glyph_actor.SetMapper( glyph_mapper)
#glyph_actor.GetProperty

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection( normals.GetOutputPort() )

actor = vtk.vtkActor()
actor.SetMapper( mapper )
actor.GetProperty().SetColor(1,1,0)

ren = vtk.vtkRenderer()
win = vtk.vtkRenderWindow()
iren = vtk.vtkRenderWindowInteractor()

ren.SetBackground(0,0,0)

win.AddRenderer( ren )
iren.SetRenderWindow( win )

ren.AddActor( actor )
ren.AddActor( glyph_actor)

iren.Render()
iren.Start()